<a href="https://colab.research.google.com/github/Plutooooo/hku_phys3151_2022/blob/main/multivariate-linear-regression/to_model_and_reveal_the_force_of_gravity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using polynomial regression to model and reveal the force of gravity

We will take a look at a simple example: the force of gravity on a falling object. The formula for the location of a falling object is:

\begin{eqnarray}
y=X_{0}+V_{0}t-\frac{1}{2}gt^{2}
\end{eqnarray}

where $X_{0}$ is the initial location of the object, i.e. its initial height.
$V_{0}$ is the initial velocity (or speed) of the object when it is at this initial position. $t$ is the time after the object was dropped. $g$ is the force of gravity, which is an acceleration of approximately 9.8 $m/s^{2}$. This is actually a constant not a variable like all of the others (unless we are dropping an object on different planets, which are not). It is negative because the force of gravity is downward, from a positive location value toward zero.

First, we will create a Python function to calculate the location of a falling object

In [ ]:
def location(x_0, v_0, t):
    return x_0 + v_0*t - (9.8/2)*t**2

Next, we use this function to create a dataset and save it in a CSV file

In [ ]:
import csv
import random
import math
random.seed
with open('gravity_location_data.csv', mode='w') as gravity_file:
    gravity_writer = csv.writer(gravity_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    gravity_writer.writerow(['initial_position', 'initial_velocity', 'mass', 'time', 'location'])
for i in range (0, 10000):
        initial_position = random.randrange(1, 10000)
        initial_velocity = random.randrange(1, 100)
        mass = random.randrange(1, 1000)
        time = random.randrange(1, 100)
gravity_writer.writerow([initial_position, initial_velocity, mass, time, location(initial_position, initial_velocity, time)])

What we are doing here is creating 10,000 examples, using randomized values for all of our variables, then calculating the location at a certain (randomly selected) time.

Now that we have our data in a CSV file, we import it

In [ ]:
import pandas as pd
gravity_data = pd.read_csv('gravity_location_data.csv')
df_location = pd.DataFrame(gravity_data)

This data will not actually be linear but scitkit-learn gives us the ability modify the data so that it can be trained as a LinearRegression model, while taking into account that it is actually a polynomial.

In [ ]:
from sklearn.model_selection import train_test_split
def split_data(data, target_name):
    y = data[target_name]
    X = data.drop(target_name, axis=1)
    return train_test_split(X, y, test_size=0.2, random_state=30)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
def train_eval_poly(X_train, X_test, y_train, y_test):
    regression_model = LinearRegression() 
    
    poly = PolynomialFeatures(degree=2)
    X_train_transform = poly.fit_transform(X_train)
    X_test_transform = poly.fit_transform(X_test)
    
    regression_model.fit(X_train_transform, y_train)
    
    print(poly.fit(X_train).get_feature_names(X_train.columns))
    
    y_pred = regression_model.predict(X_test_transform)
    print("R2: \t", r2_score(y_test, y_pred))
    print("RMSE: \t", sqrt(mean_squared_error(y_test, y_pred)))
    print("MAE: \t", mean_absolute_error(y_test, y_pred))
    
    return regression_model

Now we train the model with this code


In [ ]:
df_split = split_data(df_location, 'location')
lrModel = train_eval_poly(*df_split)

Note that the $R^{2}$ value is 1.0. Also, the root mean squared error (RMSE) and mean absolute error (MAE) are tiny. At this scale that is minuscule, essentially zero.

Now let’s look at the exponents on our model, which are the exponents on the polynomial equation the model is using to make predictions

In [ ]:
lrModel.coef_

This gives us coefficients correspond to each of the fifteen features from above. The first value (0.00000000e+00, or 0) is the coefficient for the value 1. That would be hanging constant with no variable, a constant value always added to the equation, except that the coefficient is zero so there is no constant value added to our result. The second value from our array ( 1.00000000e+00, or 1) is the coefficient on “initial_position.” And in fact, in our original equation, we have the initial velocity ($X_{0}$) with a coefficient of one.

<br>

Most of our other coefficients are essentially zero, and tell us that these factors are irrelevant to predicting the location, but we do have a few that are not. The twelfth in our list is also a coefficient of 1, which corresponds to ‘initial_velocity time,’ or initial velocity multiplied by time. Again, we have this factor in our original equation, which appears as $V_{0}t$.

<br>

Finally, the very last value is -4.9, which is the coefficient corresponding to ‘time ^ 2.’ Note that this also appears in our original equation as $\frac{1}{2}gt^{2}$. But we said earlier that $g$ is approximately 9.8, and $-\frac{1}{2} * 9.8 = -4.9$. So this also corresponds with our original equation.

<br>

There are two very interesting final points to be made here. First, the resulting model found the mass to be irrelevant to making the prediction, which is correct. Second, the machine learning algorithm was able to derive the gravitational acceleration constant g, or at least determine the value of the entire coefficient $(-\frac{1}{2} * 9.8)$ which includes $g$.